In [ ]:
from os import getcwd
import joblib
import numpy as np
import pandas as pd
from scipy.optimize import minimize
from scipy.interpolate import interp1d
from sklearn.metrics import r2_score
from sklearn.linear_model import LinearRegression
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor
from xgboost import XGBRegressor
from lightgbm import LGBMRegressor
import matplotlib.pyplot as plt
from river_framework import (
    RiverFlowDataProcessor,
    PEAK_QUANTILE,
    PATH_DF,
    PATH_FIG,
    PATH_MODEL,
)

In [ ]:
river = getcwd().split("\\")[-1]

In [ ]:
TRANSITION_WIDTH = 0.1

file_data1 = PATH_DF + "Data1_with_nan.csv"
file_data2 = PATH_DF + "Data2_without_nan.csv"

font1 = {"family": "Times New Roman", "weight": "bold", "size": 17}
font2 = {"family": "Times New Roman", "weight": "bold", "size": 15}

In [ ]:
col_list1 = ["River", "Station", "Discharge"]
col_list2 = ["River", "Station", "Discharge", "Discharge_Upper", "Discharge_Lower"]

sources = ["Observed", "Predicted", "Interpolated", "Boundary matched"]
colors = ["blue", "green", "orange", "red"]
color_dict = dict(zip(sources, colors))

# Peak Flow Correction

In [ ]:
class PeakFlowCorrection:
    def __init__(
        self, threshold, peak_quantile=PEAK_QUANTILE, transition_width=TRANSITION_WIDTH
    ):
        """
        Parameters:
            peak_quantile: float, defines the quantile threshold for peak values
            transition_width: float, proportion of the transition range width
        """
        self.peak_threshold = threshold
        self.peak_quantile = peak_quantile
        self.transition_width = transition_width
        self.correction_model = None
        self.train_metrics = {}

    def fit(self, y_true, y_pred):
        """
        Fits the corrector using training data

        Parameters:
            y_true: array-like, true values of the training set
            y_pred: array-like, predicted values of the training set
        """

        # Get peak samples
        peak_mask = y_true >= self.peak_threshold
        y_true_peaks = y_true[peak_mask]
        y_pred_peaks = y_pred[peak_mask]

        # Fit correction model
        self.correction_model = LinearRegression()
        self.correction_model.fit(y_pred_peaks.reshape(-1, 1), y_true_peaks)

        # Calculate metrics for the training set
        y_corrected = self.apply_correction(y_pred)
        self._calculate_metrics(y_true, y_pred, y_corrected)

        return self

    def apply_correction(self, y_pred):
        """Predict with smooth transition"""
        if self.correction_model is None:
            raise ValueError(
                "The model has not been trained, please call the fit method first"
            )

        y_corrected = y_pred.copy()

        # Define the transition range
        transition_lower = self.peak_threshold * (1 - self.transition_width)
        transition_upper = self.peak_threshold * (1 + self.transition_width)

        # Apply different corrections for different ranges
        for i, pred in enumerate(y_pred):
            corrected_value = self.correction_model.predict(pred.reshape(1, -1))[0]
            if pred >= transition_upper:
                # Full correction
                y_corrected[i] = corrected_value
            elif pred >= transition_lower:
                # Transition range: linear interpolation
                ratio = (pred - transition_lower) / (
                    transition_upper - transition_lower
                )
                y_corrected[i] = pred + ratio * (corrected_value - pred)
            # else: Values below the threshold remain unchanged

        return y_corrected

    def _calculate_metrics(self, y_true, y_pred, y_corrected):
        """Calculates evaluation metrics"""
        # Separate peak and non-peak samples
        peak_mask = y_true >= self.peak_threshold

        # Calculate R2
        y_true_peak = y_true[peak_mask]
        y_pred_peak = y_pred[peak_mask]
        y_corrected_peak = y_corrected[peak_mask]
        r2_peak_original = r2_score(y_true_peak, y_pred_peak)
        r2_peak_corrected = r2_score(y_true_peak, y_corrected_peak)

        r2_original = r2_score(y_true, y_pred)
        r2_corrected = r2_score(y_true, y_corrected)
        if (r2_peak_corrected > r2_peak_original) and (r2_corrected > r2_original):
            improved = True
        else:
            improved = False

        m1 = calc_metric(y_true, y_pred)
        m1.index = [f"Original_{col}" for col in m1.index]
        m2 = calc_metric(y_true, y_corrected)
        m2.index = [f"Corrected_{col}" for col in m2.index]
        m3 = calc_metric(y_true_peak, y_pred_peak)
        m3.index = [f"Original_{col}_Peak" for col in m3.index]
        m4 = calc_metric(y_true_peak, y_corrected_peak)
        m4.index = [f"Corrected_{col}_Peak" for col in m4.index]
        metric = pd.concat([m1, m2, m3, m4], axis=0)
        metric["Peak_Corrected"] = improved

        return metric

    def evaluate(self, y_true, y_pred, plot=True, final_prediction=False):
        """
        Evaluates the correction effect

        Parameters:
            y_true: array-like, true values
            y_pred: array-like, original predicted values
        """
        # Get corrected predicted values
        y_corrected = self.apply_correction(y_pred)

        # Calculate evaluation metrics
        metrics = self._calculate_metrics(y_true, y_pred, y_corrected)
        improved = metrics["Peak_Corrected"]

        # Save final predictions for the test set
        if final_prediction:
            if improved:
                y_corrected = pd.Series(y_corrected, index=y_true.index)
                y_final = pd.concat([y_true, y_corrected], axis=1)
            else:
                y_pred = pd.Series(y_pred, index=y_true.index)
                y_final = pd.concat([y_true, y_pred], axis=1)

            y_final.columns = ["Observed", "Predicted"]
            y_final.to_csv(PATH_DF + f"Final_Predictions.csv")

        # Visualize
        if plot:
            self._plot_evaluation(y_true, y_pred, y_corrected)

        # Print evaluation metrics
        print("\nEvaluation Metrics:")
        print(
            f"Peak Flow Threshold (>{self.peak_quantile*100}%): {self.peak_threshold:.2f}"
        )
        print("\nTotal Performance:")
        print(
            f"Original R2 (All): {metrics['Original_R2']:.3f} -> Corrected R2 (All): {metrics['Corrected_R2']:.3f}"
        )
        print("\nPeak Flow Performance:")
        print(
            f"Original R2 (Peak): {metrics['Original_R2_Peak']:.3f} -> Corrected R2 (Peak): {metrics['Corrected_R2_Peak']:.3f}"
        )

        return metrics

    def _plot_evaluation(self, y_true, y_pred, y_corrected):
        """Generates evaluation plots"""
        peak_mask = y_true >= self.peak_threshold

        fig, axes = plt.subplots(1, 2, figsize=(15, 7))
        # fig.suptitle("Peak Flow Correction Evaluation", fontsize=16)

        # 1. Original prediction scatter plot
        ax = axes[0]
        ax.scatter(
            y_true[~peak_mask],
            y_pred[~peak_mask],
            alpha=0.5,
            label="Normal discharge",
            color="gray",
            s=20,
        )
        ax.scatter(
            y_true[peak_mask],
            y_pred[peak_mask],
            alpha=0.5,
            label="Peak discharge",
            color="red",
            s=20,
        )
        ax.plot(
            [y_true.min(), y_true.max()],
            [y_true.min(), y_true.max()],
            "k--",
            # label="1:1 line",
        )
        ax.axhline(y=self.peak_threshold, color="r", linestyle="--", alpha=0.3)
        ax.axvline(x=self.peak_threshold, color="r", linestyle="--", alpha=0.3)
        ax.set_xlabel("Observed discharge (m³/s)", font=font2)
        ax.set_ylabel("Predicted discharge (m³/s)", font=font2)
        ax.set_title("Before Correction", font=font1)
        ax.legend(fontsize=13.5)
        plt.xticks(fontsize=13.5)
        plt.yticks(fontsize=13.5)

        # 2. Corrected scatter plot
        ax = axes[1]
        ax.scatter(
            y_true[~peak_mask],
            y_corrected[~peak_mask],
            alpha=0.5,
            label="Normal discharge",
            color="gray",
            s=20,
        )
        ax.scatter(
            y_true[peak_mask],
            y_corrected[peak_mask],
            alpha=0.5,
            label="Peak discharge",
            color="red",
            s=20,
        )
        ax.plot(
            [y_true.min(), y_true.max()],
            [y_true.min(), y_true.max()],
            "k--",
            # label="1:1 line",
        )
        ax.axhline(y=self.peak_threshold, color="r", linestyle="--", alpha=0.3)
        ax.axvline(x=self.peak_threshold, color="r", linestyle="--", alpha=0.3)
        ax.set_xlabel("Observed Flow (m³/s)", font=font2)
        ax.set_ylabel("Corrected Flow (m³/s)", font=font2)
        ax.set_title("After Correction", font=font1)
        ax.legend(fontsize=13.5)
        plt.xticks(fontsize=13.5)
        plt.yticks(fontsize=13.5)

        # plt.tight_layout()
        # plt.savefig(PATH_FIG + f"{river}_Peak_Flow_Correction.png")


def calc_metric(y_true, y_pred, decimal_digit=3, name=None):
    from sklearn.metrics import mean_absolute_error

    # Align y_true and y_pred and drop NaN values
    combined = pd.DataFrame({'y_true': y_true, 'y_pred': y_pred}).dropna()
    y_true = combined['y_true'].values
    y_pred = combined['y_pred'].values

    R = np.corrcoef(y_true, y_pred)[0, 1]
    R2 = r2_score(y_true, y_pred)
    MAE = mean_absolute_error(y_true, y_pred)
    try:
        from sklearn.metrics import mean_squared_error
        RMSE = np.sqrt(mean_squared_error(y_true, y_pred))
    except ImportError:
        from sklearn.metrics import root_mean_squared_error
        RMSE = root_mean_squared_error(y_true, y_pred)

    NRMSE = RMSE / (np.max(y_true) - np.min(y_true)) * 100
    DDR = np.mean(
        np.abs((y_pred - y_true) / np.mean(y_true))
    )  # Developed Discrepancy Ratio (custom calculation)
    IQR = np.percentile(y_true, 75) - np.percentile(y_true, 25)  # Inter-Quartile Range
    RPIQ = IQR / RMSE  # Ratio of Performance to Inter-Quartile Range

    # Calculate KGE
    alpha = np.std(y_pred) / np.std(y_true)
    beta = np.mean(y_pred) / np.mean(y_true)
    KGE = 1 - np.sqrt((R - 1) ** 2 + (alpha - 1) ** 2 + (beta - 1) ** 2)
    
    metrics = {
        "R": R,
        "R2": R2,
        "MAE": MAE,
        "RMSE": RMSE,
        "NRMSE (%)": NRMSE,
        "DDR": DDR,
        "RPIQ": RPIQ,
        "KGE": KGE,
    }
    # Convert to series
    metrics = pd.Series(metrics).round(decimal_digit)
    if name:
        metrics.name = name

    return metrics

In [ ]:
file_metric = PATH_DF + r"Metric\Model_metrics.csv"
metrics = pd.read_csv(file_metric)
metrics1 = metrics[metrics["Overfitting"] == False]
if metrics1.empty:
    best_model = metrics.loc[metrics["Score"].idxmax(), "Model"]
    augmented = metrics.loc[0, "Augmented"]
    overfitting = metrics.loc[0, "Overfitting"]
else:
    best_model = metrics1.loc[metrics1["Score"].idxmax(), "Model"]
    augmented = metrics1.iloc[0]["Augmented"]
    overfitting = metrics1.iloc[0]["Overfitting"]

In [ ]:
file_model = PATH_MODEL + f"Model\\{best_model}_model_final.pkl"
model = joblib.load(file_model)

In [ ]:
file_feature = PATH_MODEL + f"Feature\\{best_model}_selected_features.csv"
df_feature = pd.read_csv(file_feature)
features = df_feature["Feature"].tolist()

In [ ]:
data_processor = RiverFlowDataProcessor(file_data1)
data_processor.load_data()
data_processor.input_features = features
data_processor.target_variable = "Discharge"

(X_train, y_train), (X_val, y_val), (X_test, y_test) = data_processor.split_data()
threshold = np.percentile(y_train, PEAK_QUANTILE * 100)

print(y_test.shape)

In [ ]:
y_pred_train = model.predict(X_train).clip(min=0)
y_pred_test = model.predict(X_test).clip(min=0)

# Evaluation on the validation set
corrector = PeakFlowCorrection(
    threshold=threshold, peak_quantile=PEAK_QUANTILE, transition_width=TRANSITION_WIDTH
)
corrector.fit(y_train, y_pred_train)
y_pred_val = model.predict(X_val).clip(min=0)
improved = corrector.evaluate(y_val, y_pred_val, plot=False, final_prediction=False)[
    "Peak_Corrected"
]
print("\nImproved:", improved)

# Evaluation on the test set
print("-----------------------------------")
corrector = PeakFlowCorrection(
    threshold=threshold, peak_quantile=PEAK_QUANTILE, transition_width=TRANSITION_WIDTH
)
X_combined = pd.concat([X_train, X_val])
y_combined = pd.concat([y_train, y_val])
y_pred_combined = model.predict(X_combined).clip(min=0)
corrector.fit(y_combined, y_pred_combined)

In [ ]:
metrics2 = corrector.evaluate(y_test, y_pred_test, final_prediction=True)
metrics2["Peak_Corrected"] = improved
metrics2 = pd.DataFrame(metrics2).T
metrics2.insert(0, "Model", best_model)
metrics2.insert(0, "River", river)
metrics2.insert(metrics2.shape[1] - 1, "Overfitting", overfitting)
metrics2.insert(metrics2.shape[1] - 1, "Peak_Augmented", augmented)
metrics2 = metrics2.round(3)

metrics2.to_csv(PATH_DF + r"Metric\Best_model_metrics.csv", index=False)
metrics2

# Uncertainty analysis

In [ ]:
params = model.get_params()

file_train_x = PATH_MODEL + f"Set\\{best_model}_X_Train.csv"
file_train_y = PATH_MODEL + f"Set\\{best_model}_y_Train.csv"
X_train = pd.read_csv(file_train_x)
y_train = pd.read_csv(file_train_y)

if best_model == "XGBoost":
    params["objective"] = "reg:quantileerror"
    params["quantile_alpha"] = 0.975
    model1 = XGBRegressor(**params)
    model1.fit(X_train, y_train)

    params["quantile_alpha"] = 0.025
    model2 = XGBRegressor(**params)
    model2.fit(X_train, y_train)
elif best_model == "LightGBM":
    params["objective"] = "quantile"
    params["alpha"] = 0.975
    model1 = LGBMRegressor(**params)
    model1.fit(X_train, y_train)

    params["alpha"] = 0.025
    model2 = LGBMRegressor(**params)
    model2.fit(X_train, y_train)
elif best_model == "GB":
    params.pop("bootstrap", None)
    params.pop("max_samples", None)
    params.pop("monotonic_cst", None)
    params.pop("n_jobs", None)
    params.pop("oob_score", None)

    params["loss"] = "quantile"
    params["alpha"] = 0.975
    model1 = GradientBoostingRegressor(**params)
    model1.fit(X_train, y_train)

    params["alpha"] = 0.025
    model2 = GradientBoostingRegressor(**params)
    model2.fit(X_train, y_train)

# Post-processing of Time Series

In [ ]:
df = pd.read_csv(file_data1, parse_dates=["Date"], index_col="Date")

In [ ]:
# Split df to nan and non-nan
df1 = df[df.isnull().any(axis=1)]
df2 = df.dropna()[col_list1]
df2["Source"] = "Observed"

# Predict nan values
df1["Discharge"] = model.predict(df1[features])
df1["Discharge"] = df1["Discharge"].clip(lower=0)
if best_model in ["XGBoost", "LightGBM", "GB"]:
    df1["Discharge_Upper"] = model1.predict(df1[features])
    df1["Discharge_Upper"] = df1["Discharge_Upper"].clip(lower=0)
    df1["Discharge_Lower"] = model2.predict(df1[features])
    df1["Discharge_Lower"] = df1["Discharge_Lower"].clip(lower=0)
elif best_model == "RF":
    # Use the results of each tree to calculate the 2.5% and 97.5% quantiles
    n_trees = model.n_estimators
    predictions = np.zeros((df1.shape[0], n_trees))
    for i in range(n_trees):
        predictions[:, i] = model.estimators_[i].predict(df1[features])
    df1["Discharge_Upper"] = np.percentile(predictions, 97.5, axis=1)
    df1["Discharge_Lower"] = np.percentile(predictions, 2.5, axis=1)
else:
    raise ValueError("Unsupported model type")


if improved:
    df1["Discharge"] = corrector.apply_correction(df1["Discharge"].values)
    df1["Discharge_Upper"] = corrector.apply_correction(df1["Discharge_Upper"].values)
    df1["Discharge_Lower"] = corrector.apply_correction(df1["Discharge_Lower"].values)

df1 = df1[col_list2]
df1["Source"] = "Predicted"

df = pd.concat([df1, df2]).sort_index()
df

In [ ]:
discharge = df["Discharge"].values
source = df["Source"].values
dates = df.index

observed_indices = np.where(source == "Observed")[0]
predicted_indices = np.where(source == "Predicted")[0]

In [ ]:
# Get missing segments
def find_missing_segments(predicted_indices):
    segments = []
    start = predicted_indices[0]
    for i in range(1, len(predicted_indices)):
        if predicted_indices[i] != predicted_indices[i - 1] + 1:
            segments.append((start, predicted_indices[i - 1]))
            start = predicted_indices[i]
    segments.append((start, predicted_indices[-1]))
    return segments


missing_segments = find_missing_segments(predicted_indices)

In [ ]:
# Define the objective function for optimization (boundary matching)
def objective_function(
    adjustment_factors,
    discharge,
    predicted_indices,
    boundary_obs_pred,
    boundary_pred_obs,
):
    adjusted_discharge = discharge.copy()
    adjusted_discharge[predicted_indices] *= adjustment_factors

    boundary_mismatch = 0

    # Check if boundary_obs_pred is not empty and indices are valid
    if len(boundary_obs_pred) > 0 and np.all(boundary_obs_pred + 1 < len(discharge)):
        boundary_mismatch += np.sum(
            (discharge[boundary_obs_pred] - adjusted_discharge[boundary_obs_pred + 1])
            ** 2
        )

    # Check if boundary_pred_obs is not empty and indices are valid
    if len(boundary_pred_obs) > 0 and np.all(boundary_pred_obs + 1 < len(discharge)):
        boundary_mismatch += np.sum(
            (adjusted_discharge[boundary_pred_obs] - discharge[boundary_pred_obs + 1])
            ** 2
        )

    return boundary_mismatch

In [ ]:
# Process each missing segment
for start_idx, end_idx in missing_segments:
    segment_length = end_idx - start_idx + 1
    segment_dates = dates[start_idx : end_idx + 1]

    # Condition (1): Length <= 20 days and does not contain data from May to July
    if (
        segment_length <= 20
        and not ((segment_dates.month >= 5) & (segment_dates.month <= 7)).any()
    ):
        if start_idx > 0 and end_idx < len(discharge) - 1:
            # Linear interpolation
            interp_func = interp1d(
                [start_idx - 1, end_idx + 1],
                [discharge[start_idx - 1], discharge[end_idx + 1]],
                kind="linear",
            )
            discharge[start_idx : end_idx + 1] = interp_func(
                np.arange(start_idx, end_idx + 1)
            )
            source[start_idx : end_idx + 1] = "Interpolated"
        else:
            # Process the segment located at the beginning or end
            if start_idx == 0:
                # If at the beginning, fill with the right observation value
                discharge[start_idx : end_idx + 1] = discharge[end_idx + 1]
            elif end_idx == len(discharge) - 1:
                # If at the end, fill with the left observation value
                discharge[start_idx : end_idx + 1] = discharge[start_idx - 1]
            source[start_idx : end_idx + 1] = "Interpolated"

    else:
        # Condition (2): Boundary condition matching
        # Get the indices of the predicted segment
        predicted_segment_indices = np.arange(start_idx, end_idx + 1)
        boundary_obs_pred = np.array(
            [start_idx - 1] if start_idx - 1 in observed_indices else []
        )
        boundary_pred_obs = np.array(
            [end_idx + 1] if end_idx + 1 in observed_indices else []
        )

        # Initial adjustment factors for optimization
        initial_factors = np.ones(len(predicted_segment_indices))

        # Optimize the adjustment factors
        result = minimize(
            objective_function,
            initial_factors,
            args=(
                discharge,
                predicted_segment_indices,
                boundary_obs_pred,
                boundary_pred_obs,
            ),
            method="L-BFGS-B",
            bounds=[(0.2, 4)] * len(predicted_segment_indices),
        )

        # Get the length of X: 10% of the segment length and 10 days, take the minimum
        X_days = min(int(segment_length * 0.1), 10)
        X_days = max(X_days, 3)
        X_days = min(X_days, segment_length)

        # Check if there are observation data before the segment
        # If there are, adjust the first X days of the segment
        if len(boundary_obs_pred) > 0 and X_days > 0:
            discharge[start_idx : start_idx + X_days] *= result.x[:X_days]
            source[start_idx : start_idx + X_days] = "Boundary matched"

        # Check if there are observation data after the segment
        if len(boundary_pred_obs) > 0 and X_days > 0:
            discharge[end_idx - X_days + 1 : end_idx + 1] *= result.x[-X_days:]
            source[end_idx - X_days + 1 : end_idx + 1] = "Boundary matched"


df["Source"] = source
df = df.round(2)
df.to_csv(PATH_DF + "Data3_filled.csv")
df

In [ ]:
df["Source"].value_counts()

# Plot

In [ ]:
date1 = pd.Timestamp(1950, 1, 1)
date2 = pd.Timestamp(2025, 1, 1)
date_list = pd.date_range(date1, date2, freq="5YS").strftime("%Y").tolist()

In [ ]:
fig, ax = plt.subplots(figsize=(12, 6))

actual_sources = df["Source"].unique()
for source in sources:
    if source in actual_sources:
        df_source = df[df["Source"] == source]
        df_source = df_source.reindex(dates)

        ax.plot(
            df_source.index,
            df_source["Discharge"],
            label=source,
            color=color_dict[source],
        )

ax.set_xlabel("Time", font=font1)
ax.set_ylabel("Discharge (m³/s)", font=font1)

plt.xticks(date_list, date_list, fontsize=13)
plt.yticks(fontsize=13)

plt.xlim(pd.Timestamp(1949, 1, 1), pd.Timestamp(2025, 1, 1))

plt.title(f"{river} River", font=font1)

ax.legend(fontsize=13)

plt.tight_layout()
plt.savefig(PATH_FIG + f"Discharge_Time_Series_{river}.png", dpi=300)